# Text Mining

<div style="text-align">
Este <code>jupyter notebook</code> exercita algumas tarefas de mineração de texto através da biblioteca <code>NLTK</code> e avalia uma abordagem simples e determinística que permite analisar um diálogo entre operador de uma central de Chat e o cliente.
</div>

### Carregando Pacotes

In [1]:
import nltk

from nltk.text import Text
from nltk.corpus import floresta
from nltk import word_tokenize, data

### Obtendo Pacotes Específicos

In [2]:
nltk.download(['punkt', 'stopwords', 'mac_morpho', 'floresta'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Package floresta is already up-to-date!


True

### Diálogo

<div style="text-align:justify">
A seguir vamos ilustrar um diálogo entre um operador de chat e um cliente. Neste diálogo não temos a identificação das mensagens enviadas pelo operador e das mensagens enviadas pelo cliente, de modo que é preciso ler toda a conversa para chegar a conclusão de quem está falando o que.
</div>

In [3]:
dialog = [
    'Bom dia. Meu nome é Everton Gago e estou aqui te auxiliar no que precisar. Em que posso ajudar?',
    'Bom dia',
    'Eu não consigo acessar o site para consultar meu extrato',
    'Nosso site está indisponível no momento. Pedimos desculpas pelo inconveniente',
    'Isso é um absurdo',
    'O serviço de vocês é uma porcaria, nunca funciona',
    'Eu vou cancelar esse serviço',
    'Existe algo mais em que possamos ajudar?',
    'Não, obrigado!'
]

### Remover Protocolos

<div style="text-align:justify">
Para potencializar o resultado da análise é importante remover os protocolos de atendimento existentes no diálogo. Estes protocolos podem ser facilmente encontrados pois estão presentes em praticamente todos os atendimentos.
</div>

In [4]:
dialog = [msg for msg in dialog if ('Bom dia. Meu nome' not in msg) and ('em que possamos ajudar' not in msg)]
dialog

['Bom dia',
 'Eu não consigo acessar o site para consultar meu extrato',
 'Nosso site está indisponível no momento. Pedimos desculpas pelo inconveniente',
 'Isso é um absurdo',
 'O serviço de vocês é uma porcaria, nunca funciona',
 'Eu vou cancelar esse serviço',
 'Não, obrigado!']

### Extrair Tokens

In [5]:
str_dialog = ' '.join(dialog)
tokens = word_tokenize(str_dialog)
text = Text(tokens)

### Extrair Sentenças

In [6]:
sent_tokenizer = data.load("tokenizers/punkt/portuguese.pickle")
sentences = sent_tokenizer.tokenize(str_dialog)
sentences[:3]

['Bom dia Eu não consigo acessar o site para consultar meu extrato Nosso site está indisponível no momento.',
 'Pedimos desculpas pelo inconveniente Isso é um absurdo O serviço de vocês é uma porcaria, nunca funciona Eu vou cancelar esse serviço Não, obrigado!']

### Extrair Contexto da Frase

<div style="text-align:justify">
A função a seguir extraí o contexto ao entorno da palavra fornecida como parâmetro. A extração do contexto é importante para para obter mais informações sobre a situação que a palavra de interesse foi mencionada.
</div>

In [7]:
def concordance(word, context=35):
    value = ''
    for sent in sentences:
        if word in sent:
            pos = sent.index(word)
            left = ''.join(sent[:pos])
            right = ''.join(sent[pos + len(word):])
            value += f'{left[-context:]} {word} {right[:context]}'

    return value.replace('  ', ' ').strip()

concordance('site')

'Bom dia Eu não consigo acessar o site para consultar meu extrato Nosso s'

### Lista de Stop Words do NLTK

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:10]

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para']

### Remover Stop Words do Diálogo

In [9]:
words = [w.lower() for w in text if w.lower() not in stopwords]
words[:10]

['bom',
 'dia',
 'consigo',
 'acessar',
 'site',
 'consultar',
 'extrato',
 'site',
 'indisponível',
 'momento']

### Criar Dicionário de Classificação

In [10]:
def simplify_tag(t):
    if '+' in t:
        return t[t.index('+') + 1:]
    else:
        return t

twords = floresta.tagged_words()
twords = [(w.lower(), simplify_tag(t)) for (w, t) in twords]
twords[:10]

[('um', 'art'),
 ('revivalismo', 'n'),
 ('refrescante', 'adj'),
 ('o', 'art'),
 ('7_e_meio', 'prop'),
 ('é', 'v-fin'),
 ('um', 'art'),
 ('ex-libris', 'n'),
 ('de', 'prp'),
 ('a', 'art')]

### Classificar Termos do Diálogo

In [11]:
interest = {}

for w in words:
    if w not in interest:
        types = [t[1] for t in twords if t[0] == w]
        value = types[0] if len(types) > 0 else None
        interest[w] = value

In [12]:
interest.items()

dict_items([('bom', 'adj'), ('dia', 'n'), ('consigo', 'v-fin'), ('acessar', 'v-inf'), ('site', 'n'), ('consultar', None), ('extrato', None), ('indisponível', None), ('momento', 'n'), ('.', '.'), ('pedimos', 'v-fin'), ('desculpas', 'n'), ('inconveniente', 'n'), ('é', 'v-fin'), ('absurdo', 'adj'), ('serviço', 'n'), ('porcaria', None), (',', ','), ('nunca', 'adv'), ('funciona', 'v-fin'), ('vou', 'v-fin'), ('cancelar', 'v-inf'), ('obrigado', 'v-pcp'), ('!', '!')])

### Filtrar Classificação de Interesse

In [13]:
def twords_filter(tw):
    return filter(lambda x : x[1] == tw, interest.items())

### Exemplo

In [14]:
dialog = list(map(lambda k : concordance(k[0]), twords_filter('v-inf')))
' '.join(dialog)

'Bom dia Eu não consigo acessar o site para consultar meu extrato ma porcaria, nunca funciona Eu vou cancelar esse serviço Não, obrigado!'